# Building a graph in RDF using `rdflib`

First we'll build a Graph object in [`rdflib`](https://rdflib.readthedocs.io/) to which we can add nodes and relations:

In [1]:
import rdflib as rdf
import rdflib.namespace

g = rdf.Graph()

In RDF, a graph is constructed from [*triples*](https://www.w3.org/TR/n-triples/), each of which has three components:

  * *subject*: the entity being annotated
  * *predicate*: a relation between the subject and the object
  * *object*: another entity or a literal value

We'll represent the **anytime crepes** recipe by making programmatic calls to `rdflib`, starting with a URL constructed from the recipe `id` as an initial node.
We'll show that as our first subject `s` to annotate using RDF triples.

In [2]:
uri = "https://www.food.com/recipe/327593"
s = rdf.URIRef(uri)
s

rdflib.term.URIRef('https://www.food.com/recipe/327593')

Throughout work with KGs, there's an important practice of using [*persistent identifiers*](https://www.openaire.eu/what-is-a-persistent-identifier) which are both *unique* and *persistent*, in other words the opposite of [*link rot*](https://youtu.be/EEtMFq7lAKQ).

We could have used other ways to identify that node, such as a unique name.
Even so, if we think of this recipe as a resource online, then its URL is both *unique* and *persistent* as long as the "food.com" website is available. 

Next we'll use the [`rdf:type`](https://www.w3.org/TR/rdf-schema/#ch_type) for the predicate `p` to describe the subject as an instance of `wmt:Recipe`

In [3]:
from rdflib.namespace import RDF

p = RDF.type
p

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

While the first two nodes in the graph used vocabularies that are predefined in `rdflib`, now we'll need to reference other vocabularies.
We'll need to use the [`NamespaceManager`](https://rdflib.readthedocs.io/en/stable/namespaces_and_bindings.html) in `rdflib` to bind and access the namespaces for those vocabularies, which is the `nm` variable:

In [4]:
nm = g.namespace_manager

In [5]:
uri = "http://purl.org/heals/food/"
ns_wtm = rdf.Namespace(uri)

prefix = "wtm"
nm.bind(prefix, ns_wtm)

Now we can use this `wtm:` namespace to reference the object `o` as the `wtm:Recipe` entity:

In [6]:
o = ns_wtm.Recipe
o

rdflib.term.URIRef('http://purl.org/heals/food/Recipe')

Note how that object resolves to the URL <http://purl.org/heals/food/Recipe> – which is a link to the vocabulary's RDF description.

Finally, we'll add the tuple `(s, p, o,)` to the graph:

In [7]:
g.add((s, p, o,))
g

<Graph identifier=Nb33a2feb88b640449ac4a3f0f12c9b97 (<class 'rdflib.graph.Graph'>)>

Now let's add the remaining metadata for the **anytime crepes** recipe.
The required cooking time of "8 minutes" can be represented as a predicate `wtm:hasCookTime` and the literal `8` which we'll define as an `xsd:integer` value:

In [8]:
p = ns_wtm.hasCookTime
p

rdflib.term.URIRef('http://purl.org/heals/food/hasCookTime')

In [9]:
from rdflib.namespace import XSD

o = rdf.Literal("8", datatype=XSD.integer)
o

rdflib.term.Literal('8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))

In [10]:
g.add((s, p, o,))

Now let's add the three ingredients `["eggs", "milk", "whole wheat flour"]` based on the vocabulary <http://purl.org/heals/ingredient/> of food ingredients:

In [11]:
p = ns_wtm.hasIngredient
p

rdflib.term.URIRef('http://purl.org/heals/food/hasIngredient')

In [12]:
uri = "http://purl.org/heals/ingredient/"
ns_ind = rdf.Namespace(uri)

prefix = "ind"
nm.bind(prefix, ns_ind)

In [13]:
o = ns_ind.ChickenEgg
o

rdflib.term.URIRef('http://purl.org/heals/ingredient/ChickenEgg')

In [14]:
g.add((s, p, o,))

In [15]:
g.add((s, p, ns_ind.CowMilk,))
g.add((s, p, ns_ind.WholeWheatFlour,))

To confirm what we've built so far, we can iterate through each of the `(s, p, o,)` triples in the graph:

In [16]:
for s, p, o in g:
    print(s, p, o)

https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/WholeWheatFlour
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/ChickenEgg
https://www.food.com/recipe/327593 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://purl.org/heals/food/Recipe
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasCookTime 8
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/CowMilk


## Serialization as N3 triples

First let's show how to serialize the graph as `n3` triples, also known as `ttl` or [*turtle*](https://www.w3.org/TR/turtle/) format.
This will be returned from RDF as a byte array, so we'll need to use a Unicode [*codec*](https://docs.python.org/3/library/codecs.html) to convert the serialized graph into a string:

In [17]:
s = g.serialize(format="n3")
print(s.decode("utf8"))

@prefix ind: <http://purl.org/heals/ingredient/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wtm: <http://purl.org/heals/food/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://www.food.com/recipe/327593> a wtm:Recipe ;
    wtm:hasCookTime 8 ;
    wtm:hasIngredient ind:ChickenEgg,
        ind:CowMilk,
        ind:WholeWheatFlour .




Similarly, we can serialize the graph as `n3` triples to a file `tmp.ttl` in the local directory:

In [18]:
g.serialize(destination="tmp.ttl", format="n3", encoding="utf-8")

Try taking a look at the `tmp.ttl` file.
Is it the same as the serialization shown above?

## Serialization as JSON-LD

Next, let's serialize the graph in [JSON-LD](https://json-ld.org/) format, stored in the `tmp.jsonld` file in the local directory:

In [19]:
data = g.serialize(
    format="json-ld",
    indent=2,
    encoding="utf-8",
    )

with open("tmp.jsonld", "wb") as f:
    f.write(data)

Try taking a look at the `tmp.jsonld` file.
Each entity, relation, and literal value has a full URL known as an *IRI* (internationalized resource locator) which [identifies a resource](https://www.w3.org/TR/json-ld11/#iris) used to define it.

We can make these JSON-LD files a bit more succinct by adding a `context` that defines prefixes for each of the vocabularies used:

In [20]:
context = {
    "@language": "en",
    "wtm": "http://purl.org/heals/food/",
    "ind": "http://purl.org/heals/ingredient/",
    }

In [21]:
context

{'@language': 'en',
 'wtm': 'http://purl.org/heals/food/',
 'ind': 'http://purl.org/heals/ingredient/'}

Now we'll serialize again as JSON-LD, this time using the context:

In [22]:
data = g.serialize(
    format="json-ld",
    context=context,
    indent=2,
    encoding="utf-8",
    )

with open("tmp.jsonld", "wb") as f:
    f.write(data)

Open these two files and compare the difference.
Notice how the `ttl` file is easier to read (for people), while the `json-ld` file has all of the metadata explicitly linked.

---

## Exercises

**Exercise 1:**

By using `ns_ind.AllPurposeFlour` to represent `"flour"` as another possible ingredient, how would you extend the graph to represent the *German Egg Pancakes* <https://www.food.com/recipe/406738> recipe?